# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
May 8, 2020: using all images for a given batch \
May 29, 2020: Modified for new update of LBANN. File names of images changed, so new extraction code. Also added code for computing chi-squared. \



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import time
from scipy import fftpack
# from ipywidgets import interact, interact_manual,fixed, SelectMultiple, IntText, IntSlider, FloatSlider,SelectionSlider,BoundedIntText
from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/LBANN/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

[NbConvertApp] Converting notebook modules_image_analysis.ipynb to script
[NbConvertApp] Writing 15101 bytes to modules_image_analysis.py


In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
# ### Other transformatino functinos
# ### Transformation functions for image pixel values

# def f_transform_new(x):
#     if x<=50:
#         a=0.03; b=-1.0
#         return a*x+b
#     elif x>50: 
#         a=0.5/np.log(300)
#         b=0.5-a*np.log(50)
#         return a*np.log(x)+b

# def f_invtransform_new(y):
#     if y<=0.5:
#         a=0.03;b=-1.0
#         return (y-b)/a
#     elif y>0.5: 
#         a=0.5/np.log(300)
#         b=0.5-a*np.log(50)
#         return np.exp((y-b)/a)
    

# def f_transform(x):
#     return np.vectorize(f_transform_new)(x)

# def f_invtransform(s):
#     return np.vectorize(f_invtransform_new)(s)

# f_transform_new(2000)

### Modules for Extraction

In [6]:
def f_get_files_df_sorted():
    '''
    Module to create Dataframe with filenames for each epoch and step
    Sorts by step and epoch
    '''
    
    ## Get images files and .npy arrays for each image in dump_outs folder
    t1=time.time()
    files_dict={}
    keys=['train_gen','train_input','val_gen','val_input']
#     file_strg_lst=['model0-training*-gen_img*-output0.npy','model0-training*-inp_img*-output0.npy','model0-validation*-gen_img*-output0.npy','model0-validation*-inp_img*-output0.npy']
    file_strg_lst=['sgd.training*_gen_img*_output0.npy','sgd.training*_inp_img*_output0.npy','sgd.validation*_gen_img*_output0.npy','sgd.validation*_inp_img*_output0.npy']

    for key,file_strg in zip(keys,file_strg_lst):
        files_dict[key]=np.array(glob.glob(main_dir+file_strg))
        if files_dict[key].shape[0]>1000 : 
            print('Warning the number of files is very large. Possibility of memory overload')
    
    df_files=pd.DataFrame([])
    dict1={}
    t1=time.time()
    ### First get sorted Dataframe with file names
    for key in keys:
        files_arr=files_dict[key]  # Get array of files
        print(key,len(files_arr))
        for fname in files_arr:
            ### Extract the Epoch number and step number from the file name
            dict1['img_type']=key
#             dict1['epoch']=np.int32(fname.split('epoch')[-1].split('-')[0])
#             dict1['step']=np.int64(fname.split('step')[-1].split('-')[0])
            dict1['epoch']=np.int32(fname.split('epoch')[-1].split('.')[1])
            dict1['step']=np.int64(fname.split('step')[-1].split('.')[1].split('_')[0])
            dict1['fname']=fname
            
            df_files=df_files.append(dict1,ignore_index=True)
    ## Sort values
    df_files=df_files.sort_values(by=['img_type','epoch','step']).reset_index(drop=True)
    # df_files
    t2=time.time()
    print("Time for Sorting",t2-t1)
    
    return df_files


def f_filter_epoch(df_input,num_sliced=1):
    '''
    Get just the last few stored step images for each epoch
    '''
    print('Extracting last %s steps of each epoch'%(num_sliced))
    df_output=pd.DataFrame([])
    for key in ['train_gen','train_input','val_gen','val_input']: 
        ### For each type of images, get list of epochs
        df1=df_input[df_input.img_type==key]
        epochs=np.unique(df1.epoch.values).astype(int)

        for epoch in epochs:### Extract the last few steps in each epoch
            df2=df1[df1.epoch==epoch]
            df_output=df_output.append(df2.iloc[-num_sliced:])  
    
    return df_output.reset_index(drop=True)


def f_get_images_df(df_files):
    '''
    Read dataframe with file names, read files and create new dataframe with images as numpy arrays
    Also computes number of images with intensity beyond a cutoff
    '''
    
    def f_row(df_row):
        '''
        Extract image
        '''
        fname,key=df_row.fname,df_row.img_type
        a1=np.load(fname)
        if key.endswith('input'): 
            size=np.int(np.sqrt(a1.shape[-1])) ### Extract size of images (=128)
            batch_size=a1.shape[0] ### Number of batches
            samples=a1.reshape(batch_size,size,size)
        elif key.endswith('gen') : samples=a1[:,0,:,:]
        else : raise SystemError

        return samples
    
    def f_high_pixel(df_row,cutoff=0.9966):
        '''
        Get number of images with a pixel about max cut-off value
        '''
        max_arr=np.amax(df_row.images,axis=(1,2))
        num_large=max_arr[max_arr>cutoff].shape[0]

        return num_large
    
    t1=time.time()
    ##### Create new Dataframe with sorted images
    df=df_files.copy()
    df['images']=df.apply(lambda row: f_row(row), axis=1)
    t2=time.time()
    print("Time for Reading images",t2-t1)
    
    ### Store the number of images with large pixel value
    cutoff=0.9966
    df['num_large']=df.apply(lambda row: f_high_pixel(row,cutoff), axis=1)
    
    return df
    

def f_get_sample_epochs(df,img_type,start_epoch=None,end_epoch=None):
    '''
    Module to extract images for a range of epochs given a dataframe
    '''
    if start_epoch==None and end_epoch==None:
        max_epoch=np.int(np.max(df.epoch.values))
        print(max_epoch)
        start_epoch=0; end_epoch=max_epoch
#     if end_epoch==None: end_epoch=start_epoch+1
    
    arr=df[(df.epoch>=start_epoch) & (df.epoch<=end_epoch) & (df.img_type==img_type)].images.values
    arr=np.vstack(arr)
    
    return arr


def f_compute_chisqr(df):
    ''' Compute chi-sqr of pixel intensity histogram for each row
    Uses the module f_pixel_intensity to compute histograms
    '''
    
    def f_chisqr(df_row,val_hist,val_err,max_val=2000):
        ''' Compute chi-sqr of rows wrt to input data'''
        
        val_dr=val_hist.copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
#         print(val_dr)
        sample=f_invtransform(df_row.images)[0]
        ### Compute pixel histogram for row   ### !!! Ensure both pixel histograms have save bins and normalization !!! ###
        gen_hist,gen_err=f_pixel_intensity(sample,plot=False,normalize=True,bins=50,hist_range=(0,max_val),mode='avg')
        ### Compute chi-sqr
        sq_diff=(gen_hist-val_hist)**2
        ###  chi_sqr :: sum((Obs-Val)^2/(Val))
        chi_sqr1=np.sum(np.divide(sq_diff[:25],val_dr[:25]**2))
        chi_sqr2=np.sum(np.divide(sq_diff[:25],1.0))
        chi_sqr3=np.sum(gen_err[:25])/np.sum(val_err[:25])  ## measures total spread in histograms wrt to input data
        
        return chi_sqr1,chi_sqr2,chi_sqr3
    
    ### Get pixel histogram of all input data
    samples_input=f_invtransform(f_get_sample_epochs(df,'train_input'))    
    max_val=np.max(samples_input)
    val_hist,val_err=f_pixel_intensity(samples_input,plot=False,normalize=True,bins=50,hist_range=(0,max_val),mode='avg')
    del samples_input
    
    ### Get chi-sqr for each row (step-epoch) for generated data
    chi_sqrs=df.apply(lambda row: f_chisqr(row,val_hist=val_hist,val_err=val_err,max_val=2000), axis=1).values
    chi_vals=np.array(list(zip(*chi_sqrs)))  ## transposing list of list 

    df['chi_sqr1'],df['chi_sqr2'],df['chi_sqr3']=chi_vals[0],chi_vals[1],chi_vals[2]
    print(type(chi_sqrs))
    
    return df


## Extract image data 

In [7]:
fldr_name='20200529_063053_seed232_80epochs'
fldr_name='20200529_090232_seed17278_80epochs'
fldr_name='20200529_111342_seed3273_80epochs'
# fldr_name='20200601_150741_seed2020_200epochs'
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/{0}/dump_outs/trainer0/model0/'.format(fldr_name)
print(main_dir)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200529_111342_seed3273_80epochs/dump_outs/trainer0/model0/


In [8]:
### Get dataframe with file names, sorted by epoch and step
df_files=f_get_files_df_sorted()

### Slice out rows to keep only the last few steps for each epoch.
# df_files=f_filter_epoch(df_files,num_sliced=2)

#############################################################
### Read images one by one into a numpy array and create a new DataFrame
df_full=f_get_images_df(df_files)
print(df_full.shape)

# ### Filter to keep just one step per epoch
# df_full=f_filter_epoch(df,1)

Warning the number of files is very large. Possibility of memory overload
Warning the number of files is very large. Possibility of memory overload
train_gen 1202
train_input 1202
val_gen 301
val_input 301
Time for Sorting 10.013043403625488
Time for Reading images 216.58686327934265
(3006, 6)


In [ ]:
# df_full=df_full.loc[:100]

In [9]:
# df_files.head(20)
df_full[['epoch','step','img_type','fname','num_large']].iloc[[0,1,-2,-1]]

,epoch,step,img_type,fname,num_large
0,0.0,0.0,train_gen,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0
1,0.0,82.0,train_gen,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0
3004,79.0,24518.0,val_input,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0
3005,79.0,24600.0,val_input,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0


## Chi-square

In [10]:
t1=time.time()
# df1=f_compute_chisqr(df.loc[[0,1,2,3,100,200]])
df_full=f_compute_chisqr(df_full)
t2=time.time()
print("Time to compute chi-sqr",t2-t1)

79
<class 'numpy.ndarray'>
Time to compute chi-sqr 176.98170971870422


In [11]:
df_full[['chi_sqr1','chi_sqr2','chi_sqr3']].describe()

,chi_sqr1,chi_sqr2,chi_sqr3
count,3.006000e+03,3006.000000,3.006000e+03
mean,6.286650e+05,0.000052,1.028873e+02
std,3.030332e+07,0.000007,1.568004e+02
min,1.836108e+01,0.000051,1.918057e-11
25%,2.137253e+01,0.000051,6.004830e+01
50%,2.262163e+01,0.000051,8.774514e+01
75%,2.378358e+01,0.000052,1.216195e+02
max,1.655201e+09,0.000374,4.685819e+03


In [15]:
df_full.quantile(0.1,axis=1)

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.000026
6       0.000026
7       0.000026
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000026
13      0.000000
14      0.000000
15      0.000026
16      0.000026
17      0.500026
18      0.000026
19      0.000026
20      0.000026
21      0.000027
22      0.000026
23      0.000026
24      0.000026
25      0.000025
26      0.500026
27      0.000026
28      0.000026
29      0.000026
          ...   
2976    0.000026
2977    0.000026
2978    0.000026
2979    0.000026
2980    0.000026
2981    0.000026
2982    0.000026
2983    0.000026
2984    0.000026
2985    0.000026
2986    0.000026
2987    0.000026
2988    0.000026
2989    0.000026
2990    0.000026
2991    0.000026
2992    0.000026
2993    0.000026
2994    0.000026
2995    0.000026
2996    0.000025
2997    0.000026
2998    0.000026
2999    0.000026
3000    0.000026
3001    0.000026
3002    0.000026
3003    0.0000

In [17]:
df=df_full.copy()

In [ ]:
df_sliced=df[(df.chi_sqr1<30)&(df.chi_sqr3>88)][['epoch','step','chi_sqr1','chi_sqr2','chi_sqr3','img_type']]

In [ ]:
df_sliced.columns

In [ ]:
# df_sliced

def f_plot_epochs(df,mode='epoch'):
    
    fig=plt.figure(figsize=(10,3))
    # for img_type in ['val_input','val_gen','train_input','train_gen']:
    # for img_type in ['train_gen','train_input']:
    # for img_type in ['val_gen','val_input']:
    for img_type in ['train_gen']:
        df_temp=df[df.img_type==img_type]
        
        if mode=='epoch': x=df_temp.epoch.values
        elif mode=='step': x=df_temp.step.values
        print(x.shape)
        
        fig.add_subplot(1,3,1)
        plt.plot(x,df_temp['chi_sqr1'].values,linestyle='-',marker='*',label=img_type)
        plt.title('chisqr1')

        fig.add_subplot(1,3,2)
        plt.plot(x,df_temp['chi_sqr2'].values,linestyle='-',marker='*',label=img_type)
        plt.title('chisqr2')

        fig.add_subplot(1,3,3)
        plt.plot(x,df_temp['chi_sqr3'].values,linestyle='-',marker='*',label=img_type)
        plt.title('Deviation in histograms')

    plt.xlabel(mode)
    plt.legend()
    plt.tight_layout()
    
f_plot_epochs(df_sliced,mode='step')
f_plot_epochs(df_sliced,mode='epoch')

### High Pixel images

In [ ]:
### Plot number of high pixel images
# df.plot('epoch','num_large',kind='scatter')
# df=df_full.copy()
plt.figure()
plt.plot(df[df.img_type=='val_gen'].step,df[df.img_type=='val_gen'].num_large,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of large pixel images from a batch set of 128 images')

In [ ]:
df[(df.num_large>0) &(df.img_type=='val_gen')][['epoch','step','num_large','img_type']]

## Explore image samples

In [ ]:
# f_pixel_intensity(samples2,'s2',normalize=True,mode='simple',bins=50)1
# f_compare_pixel_intensity([samples2[20:60],samples4,['s2','s4'],normalize=normalize,log_scale=log_scale, mode=mode,bins=bins)
# f_compute_spectrum(samples2)
# f_compare_spectrum([samples2[20:60],samples4],['s2','s4'])

In [ ]:
def f_widget_individual(df,img_type='val_gen',idx_range=(0,50),Fig_type='pixel',normalize=True,log_scale=True,rescale=True,mode='avg'):
    '''
    Module to plot pixel intensity or power spectrum for a given sample set of images
    Options for normalization, log-scal, and rescale
    Rescale converts image pixel values from (-1,1) to the original pixel range
    2 Fig_type: pixel-> pixel intensity and spectrum -> power spectrum
    '''
    
    start,end=idx_range[0],idx_range[1]
    print('Index Range %s - %s'%(start,end))
    
    try :
        sliced_arr=f_get_sample_epochs(df,img_type=img_type,start_epoch=start,end_epoch=end)
        if sliced_arr.shape[0]<1:
            print('Input indices %s %s are invalid.\nUsing full array'%(start,end))
            start0,end=0,'end'
            sliced_arr=f_get_sample_epochs(df,img_type=img_type)
    except Exception as e:
        print(e)

    ### Crop out large pixel values
    sliced_arr=np.array([arr for arr in sliced_arr if np.max(arr)<=0.994])

    if rescale: ### Converting from pixel intensity range (-1,1) to original range
        sliced_arr=f_invtransform(sliced_arr)
    print('Array size used',sliced_arr.shape)
    
    if Fig_type=='pixel':
        f_pixel_intensity(sliced_arr,label=img_type+': {0}-{1}'.format(str(start),str(end)),normalize=normalize,log_scale=log_scale,mode=mode)
    elif Fig_type=='spectrum':
        f_compute_spectrum(sliced_arr,label=img_type+': {0}-{1}'.format(str(start),str(end)),log_scale=log_scale)



In [ ]:
interact_manual(f_widget_individual,df=fixed(df),img_type=fixed('val_gen'),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),mode=['avg','simple'],
                idx_range=IntRangeSlider(value=(0,60),min=0,max=80,step=1),)

## View image block

In [46]:
plt.clf()
f, axarr = plt.subplots(4, 4, gridspec_kw = {'wspace':0, 'hspace':0})

for i, ax in enumerate(f.axes):
    ax.grid('on', linestyle='--')
    ax.set_xticklabels([])
    ax.set_yticklabels([])

plt.show()


/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
def f_plot_grid(arr,cols=16,fig_size=(15,5)):
    ''' Plot a grid of images
    '''
    size=arr.shape[0]    
    rows=int(np.ceil(size/cols))
    print(rows,cols)
    
    fig,axarr=plt.subplots(rows,cols,figsize=fig_size, gridspec_kw = {'wspace':0, 'hspace':0})
    if rows==1: axarr=np.reshape(axarr,(rows,cols))
    if cols==1: axarr=np.reshape(axarr,(rows,cols))
    
    for i in range(min(rows*cols,size)):
        row,col=int(i/cols),i%cols
        try: 
            axarr[row,col].imshow(arr[i],origin='lower',interpolation='nearest',cmap='cool', extent = [0, 128, 0, 128])
        # Drop axis label
        except Exception as e:
            print('Exception:',e)
            pass
        temp=plt.setp([a.get_xticklabels() for a in axarr[:-1,:].flatten()], visible=False)
        temp=plt.setp([a.get_yticklabels() for a in axarr[:,1:].flatten()], visible=False)
    
#     fig.subplots_adjust(wspace=0.00,hspace=0.000)
#     fig.tight_layout()

img_arr=f_get_sample_epochs(df,'val_input',65,66)[20:50,:,:]
f_plot_grid(img_arr,cols=6,fig_size=(10,5))


5 6


/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'Figure' object has no attribute 'set_aspect'

## Compare samples

In [ ]:

def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    ### Crop out large pixel values
    for key in sample_names:
        print(sample_dict[key].shape)
        sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
        print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
        
    hist_range=(0,0.996)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        hist_range=(0,2000)
    
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale)



#### Compare different epochs

In [ ]:
# img_list,labels_list=f_get_sample_epochs(df,'val_gen',10)

img_list,labels_list=[],[]
# for epoch_range in [(20,25),(45,50),(55,60),(90,100),(150,160)]:
for epoch_range in [(i,i+2) for i in range(60,75,2)]:
# for epoch_range in [(i,i+5) for i in range(0,200,5)]:
    start,end=epoch_range[0],epoch_range[1]
    img_list.append(f_get_sample_epochs(df,'val_gen',start,end))
    labels_list.append('%s:%s'%(str(start),str(end)))

dict_samples=dict.fromkeys(labels_list)
for key,val in zip(labels_list,img_list): dict_samples[key]=val

### Compare with input
dict_samples['val input']=f_get_sample_epochs(df,img_type='val_input',start_epoch=0,end_epoch=20)
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),bins=IntText(value=50),mode=['avg','simple'])


#### Compare image types

In [ ]:
### Available options : keys=['train_gen','train_input','val_gen','val_input']
# start,end=70,75
start,end=75,80
start,end=50,55
samples1=f_get_sample_epochs(df,'val_gen',start,end)
samples2=f_get_sample_epochs(df,'val_input')
samples3=f_get_sample_epochs(df,'train_gen',start,end)
samples4=f_get_sample_epochs(df,'train_input')

print(np.max(samples1))

dict_samples={'s1':samples1, 's2': samples2, 's3': samples3, 's4':samples4}
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),bins=IntText(value=50),mode=['avg','simple'])

In [ ]:
df.shape